# Find a place to install a new japanese restaurant

## 1. Libraries to import

In [11]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

## 2. Get the position of the 20 borough of Paris

In [92]:
arr=pd.read_csv('arrondissements.csv',sep=';')
arr.drop(['C_AR','C_ARINSEE','N_SQ_AR','L_AR','N_SQ_CO','SURFACE','PERIMETRE','Geometry','LONGUEUR'],1,inplace=True)
arr.sort_values('OBJECTID',inplace=True,ignore_index=True)
arr=arr[['OBJECTID','L_AROFF','Geometry X Y']]
lat=[float(arr['Geometry X Y'][i].split(',')[0]) for i in range(0,arr.shape[0])]
long=[float(arr['Geometry X Y'][i].split(',')[1]) for i in range(0,arr.shape[0])]
arr.insert(2,'Latitude',lat)
arr.insert(3,'Longitude',long)
arr.drop(['Geometry X Y'],1,inplace=True)
arr.columns=['Number','Name','Latitude','Longitude']

In [93]:
arr

,Number,Name,Latitude,Longitude
0,1,Louvre,48.862563,2.336443
1,2,Bourse,48.868279,2.342803
2,3,Temple,48.862872,2.360001
3,4,Hôtel-de-Ville,48.854341,2.357630
4,5,Panthéon,48.844443,2.350715
5,6,Luxembourg,48.849130,2.332898
6,7,Palais-Bourbon,48.856174,2.312188
7,8,Élysée,48.872721,2.312554
8,9,Opéra,48.877164,2.337458
9,10,Entrepôt,48.876130,2.360728


## 3. Map of Paris

In [95]:
address='Paris'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Paris are 48.8566969, 2.3514616.


In [99]:
map_paris = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, n, a in zip(arr['Latitude'], arr['Longitude'], arr['Number'],arr['Name']):
    label = '{}, {}'.format(n, a)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_paris)  
    
map_paris

## 4. Connection to Foursquare

In [104]:
fs_api=pd.read_csv('foursquare.csv',header=None) #Credentials
CLIENT_ID = fs_api[0][0] # your Foursquare ID
CLIENT_SECRET = fs_api[1][0] # your Foursquare Secret
VERSION = fs_api[2][0] # Foursquare API version

In [105]:
radius = 500
LIMIT = 100

venues = []

for lat, long, n, a in zip(arr['Latitude'], arr['Longitude'], 
                           arr['Number'], arr['Name']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            n, 
            a,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [108]:
venues_df=pd.DataFrame(venues)

In [111]:
venues_df.columns=['Number','Name','Latitude','Longitude',
                   'VenueName','VenueLatitude','VenueLongitude','VenueCategory']

In [112]:
venues_df.head()

,Number,Name,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,1,Louvre,48.862563,2.336443,Musée du Louvre,48.860847,2.336440,Art Museum
1,1,Louvre,48.862563,2.336443,Palais Royal,48.863236,2.337127,Historic Site
2,1,Louvre,48.862563,2.336443,Comédie-Française,48.863088,2.336612,Theater
3,1,Louvre,48.862563,2.336443,Cour Napoléon,48.861172,2.335088,Plaza
4,1,Louvre,48.862563,2.336443,Place du Palais Royal,48.862523,2.336688,Plaza


In [115]:
venues_df.to_csv('venues_paris.csv',index=False)

## 5. Use CSV to avoid using Foursquare each time

In [116]:
venues_df=pd.read_csv('venues_paris.csv')

In [117]:
venues_df.head()

,Number,Name,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,1,Louvre,48.862563,2.336443,Musée du Louvre,48.860847,2.336440,Art Museum
1,1,Louvre,48.862563,2.336443,Palais Royal,48.863236,2.337127,Historic Site
2,1,Louvre,48.862563,2.336443,Comédie-Française,48.863088,2.336612,Theater
3,1,Louvre,48.862563,2.336443,Cour Napoléon,48.861172,2.335088,Plaza
4,1,Louvre,48.862563,2.336443,Place du Palais Royal,48.862523,2.336688,Plaza


## 6. How many venues for each borough

In [120]:
venues_df.groupby(['Number','Name']).count()

,,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Number,Name,,,,,,
1,Louvre,83,83,83,83,83,83
2,Bourse,100,100,100,100,100,100
3,Temple,85,85,85,85,85,85
4,Hôtel-de-Ville,100,100,100,100,100,100
5,Panthéon,88,88,88,88,88,88
6,Luxembourg,43,43,43,43,43,43
7,Palais-Bourbon,100,100,100,100,100,100
8,Élysée,33,33,33,33,33,33
9,Opéra,100,100,100,100,100,100


## 6. Number of unique Categories

In [121]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 197 uniques categories.


## 7. Analyze each area

In [123]:
paris_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
paris_onehot['Number'] = venues_df['Number'] 
paris_onehot['Name'] = venues_df['Name'] 

fixed_columns = list(paris_onehot.columns[-2:]) + list(paris_onehot.columns[:-2])
paris_onehot = paris_onehot[fixed_columns]

print(paris_onehot.shape)
paris_onehot.head()

(1259, 199)


,Number,Name,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auvergne Restaurant,Baby Store,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cambodian Restaurant,Canal,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Lyonese Bouchon,Market,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Optical Shop,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Resort,Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,1,Louvre,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,Louvre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,Louvre,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [124]:
paris_grouped=paris_onehot.groupby(['Number','Name']).mean().reset_index()
print(paris_grouped.shape)
paris_grouped

(20, 199)


,Number,Name,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auvergne Restaurant,Baby Store,Bakery,Bar,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cambodian Restaurant,Canal,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Donut Shop,Electronics Store,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flower Shop,Food & Drink Shop,Fountain,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewish Restaurant,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Lyonese Bouchon,Market,Mediterranean Restaurant,Memorial Site,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Okonomiyaki Restaurant,Optical Shop,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Print Shop,Pub,Ramen Restaurant,Record Shop,Resort,Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Science Museum,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Snack Place,South American Restaurant,Southwestern French Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Zoo,Zoo Exhibit
0,1,Louvre,0.000000,0.000000,0.000000,0.012048,0.000000,0.000000,0.036145,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.012048,0.000000,0.012048,0.012048,0.00,0.012048,0.00,0.000000,0.000000,0.012048,0.012048,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.036145,0.000000,0.000000,0.012048,0.024096,0.000000,0.000000,0.012048,0.012048,0.012048,0.048193,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.012048,0.000000,0.00,0.00,0.000000,0.000000,0.024096,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.108434,0.000000,0.012048,0.024096,0.000000,0.00,0.012048,0.00,0.000000,0.000000,0.000000,0.012048,0.000000,0.00000,0.000000,0.000000,0.024096,0.000000,0.000000,0.072289,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.036145,0.072289,0.00,0.000000,0.000000,0.024096,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [127]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['Number', 'Name']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Number'] = paris_grouped['Number']
neighborhoods_venues_sorted['Name'] = paris_grouped['Name']

for ind in np.arange(paris_grouped.shape[0]):
    row_categories = paris_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 2:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(20, 12)


,Number,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Louvre,French Restaurant,Hotel,Japanese Restaurant,Plaza,Coffee Shop,Café,Art Museum,Italian Restaurant,Thai Restaurant,Korean Restaurant
1,2,Bourse,French Restaurant,Wine Bar,Cocktail Bar,Hotel,Bakery,Bistro,Creperie,Coffee Shop,Concert Hall,Pastry Shop
2,3,Temple,French Restaurant,Art Gallery,Gourmet Shop,Japanese Restaurant,Sandwich Place,Bakery,Chinese Restaurant,Italian Restaurant,Cocktail Bar,Wine Bar
3,4,Hôtel-de-Ville,French Restaurant,Hotel,Ice Cream Shop,Clothing Store,Plaza,Wine Bar,Pedestrian Plaza,Garden,Pastry Shop,Park
4,5,Panthéon,French Restaurant,Italian Restaurant,Bakery,Museum,Hotel,Pub,Plaza,Coffee Shop,Café,Greek Restaurant
5,6,Luxembourg,Pastry Shop,French Restaurant,Theater,Wine Bar,Plaza,Fountain,Bakery,Playground,Bookstore,Boutique
6,7,Palais-Bourbon,Hotel,French Restaurant,Italian Restaurant,Café,Plaza,Bistro,Cocktail Bar,History Museum,Art Museum,Gourmet Shop
7,8,Élysée,French Restaurant,Hotel,Spa,Cocktail Bar,Mediterranean Restaurant,Theater,Modern European Restaurant,Corsican Restaurant,Sporting Goods Shop,Resort
8,9,Opéra,French Restaurant,Hotel,Cocktail Bar,Bistro,Bakery,Wine Bar,Italian Restaurant,Lounge,Japanese Restaurant,Pizza Place
9,10,Entrepôt,French Restaurant,Coffee Shop,Hotel,Bistro,Indian Restaurant,Café,Bar,Pizza Place,Japanese Restaurant,Wine Shop


## 8. Cluster Areas

In [135]:
kclusters = 5

paris_grouped_clustering = paris_grouped.drop(['Number', 'Name'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 1, 2, 3, 0, 4, 0, 0, 0, 0])

In [137]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
paris_merged = arr.copy()

# add clustering labels
paris_merged["Cluster Labels"] = kmeans.labels_
paris_merged = paris_merged.join(neighborhoods_venues_sorted.drop(["Name"], 1).set_index("Number"), on="Number")

print(paris_merged.shape)
paris_merged.sort_values(["Cluster Labels"], inplace=True)
paris_merged

(20, 15)


,Number,Name,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Louvre,48.862563,2.336443,0,French Restaurant,Hotel,Japanese Restaurant,Plaza,Coffee Shop,Café,Art Museum,Italian Restaurant,Thai Restaurant,Korean Restaurant
17,18,Buttes-Montmartre,48.892569,2.348161,0,Bar,French Restaurant,Convenience Store,Coffee Shop,Pizza Place,Restaurant,Middle Eastern Restaurant,Supermarket,Bistro,Fast Food Restaurant
16,17,Batignolles-Monceau,48.887327,2.306777,0,French Restaurant,Hotel,Italian Restaurant,Bakery,Plaza,Bistro,Café,Japanese Restaurant,Supermarket,Diner
14,15,Vaugirard,48.840085,2.292826,0,Italian Restaurant,Hotel,French Restaurant,Indian Restaurant,Coffee Shop,Supermarket,Thai Restaurant,Bistro,Japanese Restaurant,Park
10,11,Popincourt,48.859059,2.380058,0,French Restaurant,Restaurant,Supermarket,Wine Bar,Pastry Shop,Theater,Pizza Place,Japanese Restaurant,Italian Restaurant,Cocktail Bar
18,19,Buttes-Chaumont,48.887076,2.384821,0,French Restaurant,Bar,Hotel,Café,Bistro,Brewery,Beer Bar,Seafood Restaurant,Supermarket,Canal
8,9,Opéra,48.877164,2.337458,0,French Restaurant,Hotel,Cocktail Bar,Bistro,Bakery,Wine Bar,Italian Restaurant,Lounge,Japanese Restaurant,Pizza Place
9,10,Entrepôt,48.876130,2.360728,0,French Restaurant,Coffee Shop,Hotel,Bistro,Indian Restaurant,Café,Bar,Pizza Place,Japanese Restaurant,Wine Shop
5,6,Luxembourg,48.849130,2.332898,0,Pastry Shop,French Restaurant,Theater,Wine Bar,Plaza,Fountain,Bakery,Playground,Bookstore,Boutique
4,5,Panthéon,48.844443,2.350715,0,French Restaurant,Italian Restaurant,Bakery,Museum,Hotel,Pub,Plaza,Coffee Shop,Café,Greek Restaurant


## 9. Let's Visualize

In [142]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, num, nam, cluster in zip(paris_merged['Latitude'], paris_merged['Longitude'], 
                                             paris_merged['Number'], paris_merged['Name'], 
                                             paris_merged['Cluster Labels']):
    label = folium.Popup('{} : {} - Cluster {}'.format(num, nam, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 10. Let's Examine

### Cluster 1

In [143]:
paris_merged.loc[paris_merged['Cluster Labels'] == 0, 
                 paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Louvre,French Restaurant,Hotel,Japanese Restaurant,Plaza,Coffee Shop,Café,Art Museum,Italian Restaurant,Thai Restaurant,Korean Restaurant
17,Buttes-Montmartre,Bar,French Restaurant,Convenience Store,Coffee Shop,Pizza Place,Restaurant,Middle Eastern Restaurant,Supermarket,Bistro,Fast Food Restaurant
16,Batignolles-Monceau,French Restaurant,Hotel,Italian Restaurant,Bakery,Plaza,Bistro,Café,Japanese Restaurant,Supermarket,Diner
14,Vaugirard,Italian Restaurant,Hotel,French Restaurant,Indian Restaurant,Coffee Shop,Supermarket,Thai Restaurant,Bistro,Japanese Restaurant,Park
10,Popincourt,French Restaurant,Restaurant,Supermarket,Wine Bar,Pastry Shop,Theater,Pizza Place,Japanese Restaurant,Italian Restaurant,Cocktail Bar
18,Buttes-Chaumont,French Restaurant,Bar,Hotel,Café,Bistro,Brewery,Beer Bar,Seafood Restaurant,Supermarket,Canal
8,Opéra,French Restaurant,Hotel,Cocktail Bar,Bistro,Bakery,Wine Bar,Italian Restaurant,Lounge,Japanese Restaurant,Pizza Place
9,Entrepôt,French Restaurant,Coffee Shop,Hotel,Bistro,Indian Restaurant,Café,Bar,Pizza Place,Japanese Restaurant,Wine Shop
5,Luxembourg,Pastry Shop,French Restaurant,Theater,Wine Bar,Plaza,Fountain,Bakery,Playground,Bookstore,Boutique
4,Panthéon,French Restaurant,Italian Restaurant,Bakery,Museum,Hotel,Pub,Plaza,Coffee Shop,Café,Greek Restaurant


### Cluster 2

In [144]:
paris_merged.loc[paris_merged['Cluster Labels'] == 1, 
                 paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Reuilly,Zoo Exhibit,Zoo,Supermarket,Bike Rental / Bike Share,Park,Monument / Landmark,Dive Bar,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


### Cluster 3

In [146]:
paris_merged.loc[paris_merged['Cluster Labels'] == 2, 
                 paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Gobelins,Vietnamese Restaurant,Asian Restaurant,Chinese Restaurant,Thai Restaurant,French Restaurant,Juice Bar,Cambodian Restaurant,Metro Station,Furniture / Home Store,Butcher


### Cluster 4

In [148]:
paris_merged.loc[paris_merged['Cluster Labels'] == 3, 
                 paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Palais-Bourbon,Hotel,French Restaurant,Italian Restaurant,Café,Plaza,Bistro,Cocktail Bar,History Museum,Art Museum,Gourmet Shop
13,Observatoire,French Restaurant,Hotel,Supermarket,Pizza Place,Convenience Store,Italian Restaurant,Food & Drink Shop,Brasserie,Bakery,Bistro
7,Élysée,French Restaurant,Hotel,Spa,Cocktail Bar,Mediterranean Restaurant,Theater,Modern European Restaurant,Corsican Restaurant,Sporting Goods Shop,Resort


### Cluster 5

In [150]:
paris_merged.loc[paris_merged['Cluster Labels'] == 4, 
                 paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Passy,Park,Plaza,Lake,Bus Station,Pool,Bike Rental / Bike Share,Art Museum,Boat or Ferry,French Restaurant,Bus Stop
